![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials: Data visibility

# Visibility ![Experimental feature](images/experimental.svg)

The people using models (i.e. `System` in the case of **CoSApp**) are usually not the one who develops the model. Therefore the user may not be aware of the model limits and not all of the model parameters will be meaningful for him. 

To address those issues, **CoSApp** let the model developer specify validity range and visibility on all variables. This section will address the visibility feature.


## The concept

Three levels of visibility are available in **CoSApp**:

* `PUBLIC`: Everybody can set those variables
* `PROTECTED`: Only advanced and expert user can set those variables
* `PRIVATE`: Only expert user can set those variables

The accessibility is the intersection between the `System` tags and the `User` role:

* `PUBLIC`: no shared tag
* `PROTECTED`: more than one shared tag (but not all)
* `PRIVATE`: same tags list

![visibility](../tutorials/images/visibility.svg)

## Example

Let consider the mechanical and aerodynamic design of a blade by three users; a mechanical, a aerodynamic and a system engineer.

The `System`s for doing so will have the following inward variables:

```python
class MechanicalBlade(System):
    
    tags = ['blade', 'mechanics']
    
    def setup(self):
        self.add_inward('height', 0.4, scope=Scope.PUBLIC)
        self.add_inward('thickness', 0.01, scope=Scope.PROTECTED)
        self.add_inward('material', 'steel')  # Scope will be PRIVATE by default
        
class AerodynamicBlade(System):
    
    tags = ['blade', 'aerodynamic']
    
    def setup(self):
        self.add_inward('height', 0.4, scope=Scope.PUBLIC)
        self.add_inward('thickness', 0.01, scope=Scope.PROTECTED)
        self.add_inward('camber', 1e-3)  # Scope will be PRIVATE by default
        
class Blade(System):
    
    tags = ['blade', 'integration']
    
    def setup(self):
        self.add_child(MechanicalBlade('mechanics'), pulled={'height': 'height'})
        self.add_child(AerodynamicBlade('aerodynamic'), pulled={'height': 'height'})
```

The user will have the following role:

| User | Role |
|---|---|
| Mechanical Engineer | ["blade", "mechanics"] |
| Aerodynamic Engineer | ["blade", "aerodynamic"] |
| System Engineer | ["integration"] |


So the visibility for the variables will be:

| User | MechanicalBlade | AerodynamicBlade | Blade |
|---|---|---|---|
| Mechanical Engineer | PRIVATE | PROTECTED | PROTECTED |
| Aerodynamic Engineer | PROTECTED | PRIVATE | PROTECTED |
| System Engineer | PUBLIC | PUBLIC | PROTECTED |


## Defining visibility

The example highlight the link between the user role and its ability to set a variable.

User roles are currently saved in a configuration file (*%USERPROFILE%\\.cosapp.d\cosapp_config.json* on Windows and *\$HOME/.cosapp.d/cosapp_config.json* on Unix). A role is defined as a group of tags. For example the aerodynamic engineer will have the following roles: 

```json
{
  "roles" : [
    ["aerodynamic", "rotor"],
    ["aerodynamic", "stator"]
  ]
}
```

To determine the variable visibility for a given user, its roles will be compared to the `tags` of each `System`. If one role matches exactly the tags, the user will have the status of `PRIVATE` on the `System`. If one tag in one role matches at least one system tag, the user will have the status of `PROTECTED`. Otherwise the user will have `PUBLIC` access.

As for the validity ranges, the visibility is set in `Port` or `System` object. But it is not possible to change a `Port` variable visibility inside a `System` on the opposite of validity parameters.

For example in a `Port`:

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:
from cosapp.systems import System
from cosapp.ports import Port, Scope, ScopeError

class MyPort(Port):
    
    def setup(self):
        self.add_variable('v', 22.)  # Scope will be PUBLIC by default
        self.add_variable('w', 22., scope=Scope.PRIVATE)
        self.add_variable('x', 22., scope=Scope.PROTECTED)
        self.add_variable('y', 22., scope=Scope.PUBLIC)        

As `Port` are the doors for exchanging information between `System`, it should not be common to define a scope on them. Therefore their default visibility is `PUBLIC`.

And in a `System`:

In [ ]:
class MechanicalBlade(System):
    
    tags = ['blade', 'mechanics']  # Tags must be specified to activate visibility, otherwise all variables will be open.
    
    def setup(self):
        self.add_inward('height', 0.4, scope=Scope.PUBLIC)
        self.add_inward('thickness', 0.01, scope=Scope.PROTECTED)
        self.add_inward('material', 'steel')  # Scope will be PRIVATE by default
        
        port_in = self.add_input(MyPort, 'port_in')
        # Visibility of variable v in port_in cannot be modified
        
        port_out = self.add_output(MyPort, 'port_out')

The *inwards* are the preferred place to define variables with restrained visibility. Therefore their default scope is `PRIVATE`.

## Displaying visibility

To obtain the documentation of a *System* or a *Port*, you can use the utility function `display_doc`. 

Variables with `PRIVATE` scope will be marked with 🔒🔒 and `PROTECTED` ones with 🔒.

In [ ]:
from cosapp.tools import display_doc

display_doc(MyPort)

In [ ]:
display_doc(MechanicalBlade)

## Testing visibility

If a variable has a scope different than `PUBLIC`, user won't be able to set it except if they have the right role.

As a `System` is defining its *outputs* and *outwards* from its *inputs* and *inwards*. Only the two latter will be protected as the former will be overwrite during the component calculation.

So hoping you are not having the role ['blade', 'runner'], here are the variables you can not touch.

In [ ]:
BladeRunner = MechanicalBlade  # Class duplication
BladeRunner.tags = ['blade', 'runner']  # Changing the tags on the new class

b = BladeRunner('Ridley')

from warnings import warn

try:
    b.thickness = 0.01
except ScopeError as err:
    warn("ScopeError raised: {}".format(err))

In [ ]:
try:
    b.material = 0.01
except ScopeError as err:
    warn("ScopeError raised: {}".format(err))

In [ ]:
try:
    b.port_in.w = 0.01
except ScopeError as err:
    warn("ScopeError raised: {}".format(err))

But visibility on output port is not enforced:

In [ ]:
b.port_out.w = 0.01
print(b.port_out)